# Import dependencies

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
from census import Census

# Census API Key
from config import api_key
#change year as needed
c = Census(api_key, year=2015)

# SQLAlchemy
from sqlalchemy import create_engine
from secret import username, password 

# Store CSV into DataFrame

In [2]:
accident_csv_file = "Resources/US_Accidents_June20.csv"
accident_data_df = pd.read_csv(accident_csv_file)
accident_data_df.head()

,ID,Source,TMC,Severity,Start_Time,End_Time,Start_Lat,Start_Lng,End_Lat,End_Lng,...,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,A-1,MapQuest,201.0,3,2016-02-08 05:46:00,2016-02-08 11:00:00,39.865147,-84.058723,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Night
1,A-2,MapQuest,201.0,2,2016-02-08 06:07:59,2016-02-08 06:37:59,39.928059,-82.831184,NaN,NaN,...,False,False,False,False,False,False,Night,Night,Night,Day
2,A-3,MapQuest,201.0,2,2016-02-08 06:49:27,2016-02-08 07:19:27,39.063148,-84.032608,NaN,NaN,...,False,False,False,False,True,False,Night,Night,Day,Day
3,A-4,MapQuest,201.0,3,2016-02-08 07:23:34,2016-02-08 07:53:34,39.747753,-84.205582,NaN,NaN,...,False,False,False,False,False,False,Night,Day,Day,Day
4,A-5,MapQuest,201.0,2,2016-02-08 07:39:07,2016-02-08 08:09:07,39.627781,-84.188354,NaN,NaN,...,False,False,False,False,True,False,Day,Day,Day,Day


In [3]:
state_csv_file = "Resources/state_csvData.csv"
state_df = pd.read_csv(state_csv_file)
state_df.head()

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [4]:
Rename columns to match accident df for future merging
state_df = state_df.drop(columns={"Abbrev"})
state_df = state_df.rename(columns={"State": "state_name",
                                   "Code": "State"})
state_df.head()

,state_name,State
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


# accident_df

In [5]:
# view data before transformation
# accident_data_df.nunique()

ID                       3513617
Source                         3
TMC                           21
Severity                       4
Start_Time               3200042
End_Time                 3246120
Start_Lat                1124695
Start_Lng                1113407
End_Lat                   375074
End_Lng                   383569
Distance(mi)               13476
Description              1780092
Number                     40365
Street                    176262
Side                           3
City                       11895
County                      1724
State                         49
Zipcode                   418780
Country                        1
Timezone                       4
Airport_Code                2001
Weather_Timestamp         546086
Temperature(F)               831
Wind_Chill(F)                974
Humidity(%)                  100
Pressure(in)                1022
Visibility(mi)                85
Wind_Direction                24
Wind_Speed(mph)              160
Precipitat

In [6]:
# accident_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3513617 entries, 0 to 3513616
Data columns (total 49 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Source                 object 
 2   TMC                    float64
 3   Severity               int64  
 4   Start_Time             object 
 5   End_Time               object 
 6   Start_Lat              float64
 7   Start_Lng              float64
 8   End_Lat                float64
 9   End_Lng                float64
 10  Distance(mi)           float64
 11  Description            object 
 12  Number                 float64
 13  Street                 object 
 14  Side                   object 
 15  City                   object 
 16  County                 object 
 17  State                  object 
 18  Zipcode                object 
 19  Country                object 
 20  Timezone               object 
 21  Airport_Code           object 
 22  Weather_Timestamp 

# Create new accident df

In [7]:
new_accident_data_df = accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']].copy()
new_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy


# merge state and accident df

In [9]:
# Merge accident_df with state_df to obtain state_name for future merging with population df
clean_accident_data_df = pd.merge(new_accident_data_df, state_df, on= "State")
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


In [10]:
# convert start_time from object to datetime 
clean_accident_data_df['Start_Time'] = pd.to_datetime(clean_accident_data_df['Start_Time'])
clean_accident_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3513617 entries, 0 to 3513616
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   ID                 object        
 1   Start_Time         datetime64[ns]
 2   City               object        
 3   County             object        
 4   State              object        
 5   Zipcode            object        
 6   Street             object        
 7   Timezone           object        
 8   Temperature(F)     float64       
 9   Visibility(mi)     float64       
 10  Weather_Condition  object        
 11  state_name         object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 348.5+ MB


In [11]:
# clean_accident_data_df.nunique()

ID                   3513617
Start_Time           3200042
City                   11895
County                  1724
State                     49
Zipcode               418780
Street                176262
Timezone                   4
Temperature(F)           831
Visibility(mi)            85
Weather_Condition        127
state_name                49
dtype: int64

In [12]:
# Limit data from 2016-2020 to 2016 only
clean_accident_data_df = clean_accident_data_df[(clean_accident_data_df['Start_Time']>= "2016-01-01") & 
                                            (clean_accident_data_df['Start_Time']<= "2018-12-31")]
# clean_accident_data_df.nunique()

ID                   2018197
Start_Time           1875832
City                   10793
County                  1658
State                     49
Zipcode               265911
Street                128158
Timezone                   4
Temperature(F)           780
Visibility(mi)            71
Weather_Condition         95
state_name                49
dtype: int64

In [13]:
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
2,A-3,2016-02-08 06:49:27,Williamsburg,Clermont,OH,45176,State Route 32,US/Eastern,36.0,10.0,Overcast,Ohio
3,A-4,2016-02-08 07:23:34,Dayton,Montgomery,OH,45417,I-75 S,US/Eastern,35.1,9.0,Mostly Cloudy,Ohio
4,A-5,2016-02-08 07:39:07,Dayton,Montgomery,OH,45459,Miamisburg Centerville Rd,US/Eastern,36.0,6.0,Mostly Cloudy,Ohio


# Check to see if all 2016 data is present

In [15]:
# Sort by datetime
clean_accident_data_df = clean_accident_data_df.sort_values('Start_Time')
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain,Ohio
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,45203,I-75 S,US/Eastern,36.0,10.0,Overcast,Ohio


In [17]:
# clean_accident_data_df.count()

ID                   2018197
Start_Time           2018197
City                 2018133
County               2018197
State                2018197
Zipcode              2017659
Street               2018197
Timezone             2016487
Temperature(F)       1979434
Visibility(mi)       1971104
Weather_Condition    1970670
state_name           2018197
dtype: int64

In [18]:
# clean_accident_data_df.nunique()

ID                   2018197
Start_Time           1875832
City                   10793
County                  1658
State                     49
Zipcode               265911
Street                128158
Timezone                   4
Temperature(F)           780
Visibility(mi)            71
Weather_Condition         95
state_name                49
dtype: int64

# Test to see if correct data is included

In [23]:
# clean_accident_data_df['normalised_date'] = clean_accident_data_df['Start_Time'].dt.normalize()
# clean_accident_data_df.head()

,ID,Start_Time,City,County,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition,state_name,normalised_date
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain,Ohio,2016-02-08
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio,2016-02-08
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain,Ohio,2016-02-08
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain,Ohio,2016-02-08
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,45203,I-75 S,US/Eastern,36.0,10.0,Overcast,Ohio,2016-02-08


# Reorganize df to look pretty

In [25]:
clean_accident_data_df = clean_accident_data_df[['ID', 'Start_Time', 'City', 'County', 'State', 'state_name', 'Zipcode', 'Street', 'Timezone', 'Temperature(F)', 'Visibility(mi)', 'Weather_Condition']]
clean_accident_data_df.head()

,ID,Start_Time,City,County,State,state_name,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [28]:
clean_accident_data_df = clean_accident_data_df.rename(columns={"State": "state_abbrev",
                                                               "state_name":"State",
                                                               "Start_Time": "Date_Time"})


#  Population from 2015-2018

In [32]:
#2015
c = Census(api_key, year=2015)
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})
county_pop15_df = pd.DataFrame(county_pop_data)
county_pop15_df.rename(columns = {"B01001_001E":"2015"}, inplace=True)
county_pop15_df.head()

,NAME,2015,state,county
0,"Bibb County, Alabama",22604.0,01,007
1,"Blount County, Alabama",57710.0,01,009
2,"Bullock County, Alabama",10678.0,01,011
3,"Butler County, Alabama",20354.0,01,013
4,"Calhoun County, Alabama",116648.0,01,015


In [34]:
#2016
c = Census(api_key, year=2016)
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})
county_pop16_df = pd.DataFrame(county_pop_data)
county_pop16_df.rename(columns = {"B01001_001E":"2016"}, inplace=True)
county_pop16_df.head()

,NAME,2016,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [36]:
#2017
c = Census(api_key, year=2017)
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})
county_pop17_df = pd.DataFrame(county_pop_data)
county_pop17_df.rename(columns = {"B01001_001E":"2017"}, inplace=True)
county_pop17_df.head()

,NAME,2017,state,county
0,"Corozal Municipio, Puerto Rico",34933,72,047
1,"Maunabo Municipio, Puerto Rico",11297,72,095
2,"Peñuelas Municipio, Puerto Rico",21661,72,111
3,"Ponce Municipio, Puerto Rico",148863,72,113
4,"San Sebastián Municipio, Puerto Rico",38970,72,131


In [38]:
#2018
c = Census(api_key, year=2018)
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})
county_pop18_df = pd.DataFrame(county_pop_data)
county_pop18_df.rename(columns = {"B01001_001E":"2018"}, inplace=True)
county_pop18_df.head()

,NAME,2018,state,county
0,"Washington County, Mississippi",47086.0,28,151
1,"Perry County, Mississippi",12028.0,28,111
2,"Choctaw County, Mississippi",8321.0,28,019
3,"Itawamba County, Mississippi",23480.0,28,057
4,"Carroll County, Mississippi",10129.0,28,015


In [39]:
df_1516 = pd.merge(county_pop15_df, county_pop16_df, on="NAME")

In [40]:
df_1718 = pd.merge(county_pop17_df, county_pop18_df, on="NAME")

In [41]:
county_pop_df = pd.merge(df_1516, df_1718, on="NAME")

In [42]:
county_pop_df.head()

,NAME,2015,state_x_x,county_x_x,2016,state_y_x,county_y_x,2017,state_x_y,county_x_y,2018,state_y_y,county_y_y
0,"Bibb County, Alabama",22604.0,01,007,22572,01,007,22580,01,007,22527.0,01,007
1,"Blount County, Alabama",57710.0,01,009,57704,01,009,57667,01,009,57645.0,01,009
2,"Bullock County, Alabama",10678.0,01,011,10552,01,011,10478,01,011,10352.0,01,011
3,"Butler County, Alabama",20354.0,01,013,20280,01,013,20126,01,013,20025.0,01,013
4,"Calhoun County, Alabama",116648.0,01,015,115883,01,015,115527,01,015,115098.0,01,015


In [43]:
county_pop_df.drop(columns=["state_x_x","county_x_x","state_y_x","county_y_x","state_x_y","county_x_y"], inplace=True)


In [44]:
county_pop_df.rename(columns = {"NAME":"county", "state_y_y":"state_id", "county_y_y":"county_id"}, inplace=True)

In [45]:
county_pop_df = county_pop_df[["county","county_id","state_id","2015","2016","2017","2018"]]
county_pop_df.head()

,county,county_id,state_id,2015,2016,2017,2018
0,"Bibb County, Alabama",007,01,22604.0,22572,22580,22527.0
1,"Blount County, Alabama",009,01,57710.0,57704,57667,57645.0
2,"Bullock County, Alabama",011,01,10678.0,10552,10478,10352.0
3,"Butler County, Alabama",013,01,20354.0,20280,20126,20025.0
4,"Calhoun County, Alabama",015,01,116648.0,115883,115527,115098.0


In [48]:
#make another call for state population just to get a list of states since county infor only have state census id
state_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "state:*"})

In [49]:
#pass on the state pop data to a df
state_pop_df = pd.DataFrame(state_pop_data)
state_pop_df.head()

,NAME,B01001_001E,state
0,Minnesota,5527358.0,27
1,Mississippi,2988762.0,28
2,Missouri,6090062.0,29
3,Montana,1041732.0,30
4,Nebraska,1904760.0,31


In [50]:
#drop population column from the state pop data df
state_pop1_df = state_pop_df.drop(columns=["B01001_001E"])

In [51]:
state_pop1_df.head()

,NAME,state
0,Minnesota,27
1,Mississippi,28
2,Missouri,29
3,Montana,30
4,Nebraska,31


In [52]:
#rename "Name" column to prevent duplication when merging with county pop df
state_pop1_df.rename(columns = {"NAME":"state", "state":"state_id"}, inplace=True)

In [53]:
state_pop1_df.head()

,state,state_id
0,Minnesota,27
1,Mississippi,28
2,Missouri,29
3,Montana,30
4,Nebraska,31


In [54]:
#merge state and county df on the "state" the numeric identifier for states
county_pop1_df = pd.merge(county_pop_df, state_pop1_df, on="state_id")

In [55]:
county_pop1_df.head()

,county,county_id,state_id,2015,2016,2017,2018,state
0,"Bibb County, Alabama",007,01,22604.0,22572,22580,22527.0,Alabama
1,"Blount County, Alabama",009,01,57710.0,57704,57667,57645.0,Alabama
2,"Bullock County, Alabama",011,01,10678.0,10552,10478,10352.0,Alabama
3,"Butler County, Alabama",013,01,20354.0,20280,20126,20025.0,Alabama
4,"Calhoun County, Alabama",015,01,116648.0,115883,115527,115098.0,Alabama


In [56]:
#reorder columns 
county_pop1_df = county_pop1_df[["county","county_id","state","state_id","2015","2016","2017","2018"]]

In [58]:
#convert population columns to numeric
county_pop1_df[["2015","2016","2017","2018"]] = county_pop1_df[["2015","2016","2017","2018"]].apply(pd.to_numeric)

In [59]:
county_pop1_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3203 entries, 0 to 3202
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   county     3203 non-null   object 
 1   county_id  3203 non-null   object 
 2   state      3203 non-null   object 
 3   state_id   3203 non-null   object 
 4   2015       3203 non-null   float64
 5   2016       3203 non-null   int64  
 6   2017       3203 non-null   int64  
 7   2018       3203 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 225.2+ KB


In [60]:
#import US state 2 letter abbreviation dictionary to create a state table 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [61]:
state_series = pd.Series(us_state_abbrev)

In [63]:
state_df = state_series.to_frame()

In [65]:
state_df.reset_index(inplace=True)

In [67]:
state_df.rename(columns={"index":"state",0:"statecode"}, inplace=True)

In [68]:
state1_df = pd.merge(state_df,state_pop1_df, on="state" )

In [69]:
#County Estimated Population 2015 to 2018
county_pop1_df.head()

,county,county_id,state,state_id,2015,2016,2017,2018
0,"Bibb County, Alabama",007,Alabama,01,22604.0,22572,22580,22527.0
1,"Blount County, Alabama",009,Alabama,01,57710.0,57704,57667,57645.0
2,"Bullock County, Alabama",011,Alabama,01,10678.0,10552,10478,10352.0
3,"Butler County, Alabama",013,Alabama,01,20354.0,20280,20126,20025.0
4,"Calhoun County, Alabama",015,Alabama,01,116648.0,115883,115527,115098.0


In [70]:
#STATE TABLE
state1_df.head()

,state,statecode,state_id
0,Alabama,AL,01
1,Alaska,AK,02
2,Arizona,AZ,04
3,Arkansas,AR,05
4,California,CA,06


# edit population

In [71]:
county_pop1_df.rename(columns = {"2015":"2015 Population", 
                                "2016":"2016 Population",
                                "2017":"2017 Population",
                                "2018":"2018 Population"}, inplace=True)
county_pop1_df.head(300)

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,"Bibb County, Alabama",007,Alabama,01,22604.0,22572,22580,22527.0
1,"Blount County, Alabama",009,Alabama,01,57710.0,57704,57667,57645.0
2,"Bullock County, Alabama",011,Alabama,01,10678.0,10552,10478,10352.0
3,"Butler County, Alabama",013,Alabama,01,20354.0,20280,20126,20025.0
4,"Calhoun County, Alabama",015,Alabama,01,116648.0,115883,115527,115098.0
...,...,...,...,...,...,...,...,...
295,"Rock Island County, Illinois",161,Illinois,17,147161.0,146536,146205,145275.0
296,"St. Clair County, Illinois",163,Illinois,17,267029.0,265569,264433,263463.0
297,"Saline County, Illinois",165,Illinois,17,24783.0,24659,24430,24231.0
298,"Sangamon County, Illinois",167,Illinois,17,199016.0,198639,198134,197661.0


In [72]:
clean_pop_df = county_pop1_df.copy()
clean_pop_df.head()

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,"Bibb County, Alabama",007,Alabama,01,22604.0,22572,22580,22527.0
1,"Blount County, Alabama",009,Alabama,01,57710.0,57704,57667,57645.0
2,"Bullock County, Alabama",011,Alabama,01,10678.0,10552,10478,10352.0
3,"Butler County, Alabama",013,Alabama,01,20354.0,20280,20126,20025.0
4,"Calhoun County, Alabama",015,Alabama,01,116648.0,115883,115527,115098.0


In [73]:
# Cleaned county column
clean_pop_df['county'] = clean_pop_df['county'].str.split(",", expand=True)[0]
clean_pop_df['county'] = clean_pop_df['county'].str.replace('County', '')
clean_pop_df.head()

,county,county_id,state,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Bibb,007,Alabama,01,22604.0,22572,22580,22527.0
1,Blount,009,Alabama,01,57710.0,57704,57667,57645.0
2,Bullock,011,Alabama,01,10678.0,10552,10478,10352.0
3,Butler,013,Alabama,01,20354.0,20280,20126,20025.0
4,Calhoun,015,Alabama,01,116648.0,115883,115527,115098.0


In [75]:
# clean_pop_df = clean_pop_df.drop(columns={"state"})
clean_pop_df = clean_pop_df.rename(columns={"county": "County",
                                           "state": "State"})
clean_pop_df.head()

,County,county_id,State,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Bibb,007,Alabama,01,22604.0,22572,22580,22527.0
1,Blount,009,Alabama,01,57710.0,57704,57667,57645.0
2,Bullock,011,Alabama,01,10678.0,10552,10478,10352.0
3,Butler,013,Alabama,01,20354.0,20280,20126,20025.0
4,Calhoun,015,Alabama,01,116648.0,115883,115527,115098.0


## Extract,Clean,Transform DL data 

In [76]:
#Extract Licensed Drivers by state (DL) csv into pandas DF for year 2014,2015,2016
DL_DF = pd.read_csv("Resources/DL.csv", usecols=[0,66,67,68])
DL_DF

,STATE,2014,2015,2016
0,Alabama,"3,881,542","3,907,038","3,943,082"
1,Alaska,"531,744","533,227","534,585"
2,Arizona,"4,881,801","4,978,762","5,082,305"
3,Arkansas,"2,111,873","2,119,578","2,391,103"
4,California,"24,813,346","25,532,920","26,199,436"
5,Colorado,"3,883,362","3,974,521","4,066,580"
6,Connecticut,"2,542,588","2,566,673","2,611,007"
7,Delaware,"732,349","742,524","756,328"
8,Dist. of Col.,"419,896","455,602","489,831"
9,Florida,"13,898,347","14,262,715","14,675,160"


In [77]:
# clean Licensed Drivers by state (DL)
DL_DF_Clean = DL_DF.rename(columns={'2014':'num_licenced_driver_2014',
                                   '2015':'num_licenced_driver_2015',
                                   '2016':'num_licenced_driver_2016',
                                    'STATE':'state_name'})
DL_DF_Clean                       

,state_name,num_licenced_driver_2014,num_licenced_driver_2015,num_licenced_driver_2016
0,Alabama,"3,881,542","3,907,038","3,943,082"
1,Alaska,"531,744","533,227","534,585"
2,Arizona,"4,881,801","4,978,762","5,082,305"
3,Arkansas,"2,111,873","2,119,578","2,391,103"
4,California,"24,813,346","25,532,920","26,199,436"
5,Colorado,"3,883,362","3,974,521","4,066,580"
6,Connecticut,"2,542,588","2,566,673","2,611,007"
7,Delaware,"732,349","742,524","756,328"
8,Dist. of Col.,"419,896","455,602","489,831"
9,Florida,"13,898,347","14,262,715","14,675,160"


## Extract,Clean,Transform RMV data 

In [78]:
#Extract Registered Motor Vehicles(RMV) csv into pandas DF
RMV_DF = pd.read_csv("Resources/RMV.csv", usecols=[0,15], skiprows=4)
RMV_DF

,Unnamed: 0,Unnamed: 15
0,Alabama,"5,468,301"
1,Alaska,"794,614"
2,Arizona,"5,786,891"
3,Arkansas,"2,808,138"
4,California,"30,221,033"
5,Colorado,"5,116,341"
6,Connecticut,"2,841,842"
7,Delaware,"1,003,840"
8,Dist. of Col.,"337,100"
9,Florida,"16,600,317"


In [79]:
# clean egistered Motor Veicheles(RMV)
RMV_DF_Clean = RMV_DF.rename(columns={'Unnamed: 0':'state_name','Unnamed: 15':'num_reg_vehicle'})
RMV_DF_Clean

,state_name,num_reg_vehicle
0,Alabama,"5,468,301"
1,Alaska,"794,614"
2,Arizona,"5,786,891"
3,Arkansas,"2,808,138"
4,California,"30,221,033"
5,Colorado,"5,116,341"
6,Connecticut,"2,841,842"
7,Delaware,"1,003,840"
8,Dist. of Col.,"337,100"
9,Florida,"16,600,317"


# Export csv 

In [80]:
RMV_DF_Clean.head()

,state_name,num_reg_vehicle
0,Alabama,"5,468,301"
1,Alaska,"794,614"
2,Arizona,"5,786,891"
3,Arkansas,"2,808,138"
4,California,"30,221,033"


In [124]:
RMV_DF_Clean.to_csv("Resources/Output/reg_vehicle.csv", index=False)

In [82]:
DL_DF_Clean.head()

,state_name,num_licenced_driver_2014,num_licenced_driver_2015,num_licenced_driver_2016
0,Alabama,"3,881,542","3,907,038","3,943,082"
1,Alaska,"531,744","533,227","534,585"
2,Arizona,"4,881,801","4,978,762","5,082,305"
3,Arkansas,"2,111,873","2,119,578","2,391,103"
4,California,"24,813,346","25,532,920","26,199,436"


In [125]:
DL_DF_Clean.to_csv("Resources/Output/license.csv", index=False)

In [84]:
clean_pop_df.head()

,County,county_id,State,state_id,2015 Population,2016 Population,2017 Population,2018 Population
0,Bibb,007,Alabama,01,22604.0,22572,22580,22527.0
1,Blount,009,Alabama,01,57710.0,57704,57667,57645.0
2,Bullock,011,Alabama,01,10678.0,10552,10478,10352.0
3,Butler,013,Alabama,01,20354.0,20280,20126,20025.0
4,Calhoun,015,Alabama,01,116648.0,115883,115527,115098.0


In [102]:
clean_pop_df_rename = clean_pop_df.copy()
clean_pop_df_rename = clean_pop_df_rename.rename(columns= {"2015 Population": "population_2015",
                                   "2016 Population": "population_2016",
                                   "2017 Population": "population_2017",
                                   "2018 Population": "population_2018",
                                   "County": "county"})


In [104]:
clean_pop_df_rename = clean_pop_df_rename.sort_values(by='State')

In [106]:
clean_pop_df_rename.drop(columns= ["State", "state_id", "county_id"], inplace=True)

In [108]:
clean_pop_df_rename= clean_pop_df_rename[["county", "population_2015", "population_2016","population_2017", "population_2018"]]
clean_pop_df_rename.head()

,county,population_2015,population_2016,population_2017,population_2018
0,Bibb,22604.0,22572,22580,22527.0
36,Lawrence,33586.0,33433,33288,33171.0
37,Lee,150982.0,153947,156597,159287.0
38,Limestone,88805.0,90257,91695,93052.0
39,Lowndes,10742.0,10565,10362,10236.0


In [109]:
# clean_pop_df_rename.nunique()

county             1938
population_2015    3147
population_2016    3147
population_2017    3158
population_2018    3143
dtype: int64

In [128]:
clean_pop_df_rename.to_csv("Resources/Output/population.csv", index=False)

In [113]:
clean_accident_data_df.head()

,ID,Date_Time,City,County,state_abbrev,State,Zipcode,Street,Timezone,Temperature(F),Visibility(mi),Weather_Condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,OH,Ohio,43017,Outerbelt E,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,OH,Ohio,45424,I-70 E,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,OH,Ohio,43068-3402,Brice Rd,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,OH,Ohio,45203,I-75 S,US/Eastern,36.0,10.0,Overcast


In [114]:
clean_accident_data_df_rename = clean_accident_data_df.copy()
clean_accident_data_df_rename = clean_accident_data_df_rename.rename(columns= {"ID": "id",
                                                                              "Date_Time": "datetime",
                                                                              "City": "city",
                                                                              "County": "county",
                                                                              "State": "state_name",
                                                                              "Zipcode": "zipcode",
                                                                              "Timezone": "timezone",
                                                                              "Temperature(F)": "temperature",
                                                                               "Visibility(mi)": "visibility",
                                                                               "Weather_Condition": "weather_condition"})
# drop street and state abbrev


In [115]:
clean_accident_data_df_rename.drop(columns= ["state_abbrev", "Street"], inplace=True)

In [116]:
clean_accident_data_df_rename.head()

,id,datetime,city,county,state_name,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,Ohio,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,Ohio,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,Ohio,45203,US/Eastern,36.0,10.0,Overcast


In [117]:
clean_accident_data_df_rename = clean_accident_data_df_rename.sort_values(by='datetime')
clean_accident_data_df_rename.head()

,id,datetime,city,county,state_name,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Dublin,Franklin,Ohio,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Dayton,Montgomery,Ohio,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Reynoldsburg,Franklin,Ohio,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Cincinnati,Hamilton,Ohio,45203,US/Eastern,36.0,10.0,Overcast


In [118]:
clean_accident_data_df_rename = clean_accident_data_df_rename[['id', 'datetime', 'state_name', 'city', 'county', 'zipcode', 'timezone', 'temperature', 'visibility', 'weather_condition']]
clean_accident_data_df_rename.head()

,id,datetime,state_name,city,county,zipcode,timezone,temperature,visibility,weather_condition
45629,A-2478859,2016-02-08 00:37:08,Ohio,Dublin,Franklin,43017,US/Eastern,42.1,10.0,Light Rain
0,A-1,2016-02-08 05:46:00,Ohio,Dayton,Montgomery,45424,US/Eastern,36.9,10.0,Light Rain
45630,A-2478860,2016-02-08 05:56:20,Ohio,Dayton,Montgomery,45424,US/Eastern,36.9,10.0,Light Rain
1,A-2,2016-02-08 06:07:59,Ohio,Reynoldsburg,Franklin,43068-3402,US/Eastern,37.9,10.0,Light Rain
45631,A-2478861,2016-02-08 06:15:39,Ohio,Cincinnati,Hamilton,45203,US/Eastern,36.0,10.0,Overcast


In [119]:
# clean_accident_data_df_rename.nunique()

id                   2018197
datetime             1875832
state_name                49
city                   10793
county                  1658
zipcode               265911
timezone                   4
temperature              780
visibility                71
weather_condition         95
dtype: int64

In [126]:
clean_accident_data_df_rename.to_csv("Resources/Output/accident.csv", index=False)

In [3]:
rds_connection_string =f'{username}:{password}@localhost:5432/us_accidents

SyntaxError: EOL while scanning string literal (<ipython-input-3-bd8a742d2fd5>, line 1)

In [ ]:
engine.table_names()

In [ ]:
RMV_DF_Clean_data_df.to_sql(name='reg_vehicle', con=engine, if_exists='append', index=False)

In [ ]:
DL_DF_Clean.to_sql(name='license', con=engine, if_exists='append', index=False)

In [ ]:
clean_pop_df_rename.to_sql(name='population', con=engine, if_exists='append', index=False)

In [ ]:
clean_accident_data_df_rename.to_sql(name='accident', con=engine, if_exists='append', index=False)